---
title: "Eastern Scheldt Barrier"
execute:
  echo: false
---




# Overview

This document provides a complete analysis workflow for the Eastern Scheldt barrier system, combining barrier closure analysis, tide gauge data processing, tidal analysis, and visualization. The analysis covers the period from 1986 to 2024 and includes:

1. **Barrier Closure Analysis**: Loading and analyzing past barrier closure dates, counting closures per water year, and calculating statistics
2. **Tide Gauge Data Processing**: Loading and combining tide gauge data from multiple sources (OS4 and RPBU gauges)
3. **Tidal Analysis**: Performing harmonic tidal decomposition and generating predicted astronomical tides
4. **Visualization**: Creating comprehensive visualizations showing predicted high waters, observed water levels, and barrier closures

Water years run from July 1 to June 30, which is more appropriate for coastal flood analysis than calendar years.


In [ ]:
#| label: shared-utilities
#| include: true

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os

# Shared configuration
output_dir = 'output'
data_dir = '../2_DATA'
os.makedirs(output_dir, exist_ok=True)

# Spatial bounds for maps
XB_GTSM = [-10, 10]  # GTSM longitude bounds
YB_GTSM = [45, 60]   # GTSM latitude bounds
XB_MET = [-30, 15]   # ERA5 longitude bounds
YB_MET = [40, 70]    # ERA5 latitude bounds

# Time intervals for spatial plots
TINT = 6   # Time interval for GTSM plots (hours)
TINT_MET = 12  # Time interval for meteorological plots (hours)

def load_master1_data():
    """Load barrier closure data from mast1.pkl"""
    mast1_file = os.path.join(output_dir, 'mast1.pkl')
    if not os.path.exists(mast1_file):
        raise FileNotFoundError(
            f"Required file {mast1_file} not found. "
            "Please run Part 1 (Barrier Closure Analysis) first to generate the required data file."
        )
    with open(mast1_file, 'rb') as f:
        data = pickle.load(f)
    return data['OCD']  # Observed closure dates

def load_master3_data():
    """Load tide gauge and tidal prediction data from mast3.pkl"""
    mast3_file = os.path.join(output_dir, 'mast3.pkl')
    if not os.path.exists(mast3_file):
        raise FileNotFoundError(
            f"Required file {mast3_file} not found. "
            "Please run Part 1, Part 2, and Part 3 first to generate the required data files."
        )
    with open(mast3_file, 'rb') as f:
        data = pickle.load(f)
    return {
        'TSP': np.array(data['TSP']),
        'TIP': np.array(data['TIP']),
        'WLP': np.array(data['WLP'])
    }

def load_all_data():
    """Load both master1 and master3 data, return as dict"""
    OCD = load_master1_data()
    OCD = np.array(OCD)
    data3 = load_master3_data()
    return {
        'OCD': OCD,
        'TSP': data3['TSP'],
        'TIP': data3['TIP'],
        'WLP': data3['WLP']
    }

def create_storms_df(OCD):
    """Create storms DataFrame from closure dates"""
    storms_df = pd.DataFrame({
        'Closure Date': [d.strftime('%Y-%m-%d %H:%M') if isinstance(d, datetime) else str(d) for d in OCD],
        'Year': [d.year if isinstance(d, datetime) else None for d in OCD],
        'Month': [d.strftime('%B') if isinstance(d, datetime) else None for d in OCD],
        'YearMonth': [(d.year, d.month) if isinstance(d, datetime) else None for d in OCD]
    })
    
    # Add water year information
    def get_water_year(date):
        """Determine water year (July 1 to June 30)"""
        if isinstance(date, datetime):
            if date.month >= 7:
                return f"{date.year}/{str(date.year + 1)[2:]}"
            else:
                return f"{date.year - 1}/{str(date.year)[2:]}"
        return None
    
    storms_df['Water Year'] = [get_water_year(d) for d in OCD]
    
    # Create Event index that groups storms by year and month
    unique_year_months = storms_df['YearMonth'].unique()
    year_month_to_index = {ym: idx + 1 for idx, ym in enumerate(sorted(unique_year_months))}
    storms_df['Event'] = storms_df['YearMonth'].map(year_month_to_index)
    
    # Reorder columns
    storms_df = storms_df[['Event', 'Closure Date', 'Water Year', 'Year', 'Month']]
    
    return storms_df

def get_or_create_storms_df():
    """Get storms_df from pickle file or create it if it doesn't exist"""
    storms_df_file = os.path.join(output_dir, 'storms_df.pkl')
    
    if os.path.exists(storms_df_file):
        with open(storms_df_file, 'rb') as f:
            return pickle.load(f)
    else:
        # Load OCD and create storms_df
        OCD = load_master1_data()
        storms_df = create_storms_df(OCD)
        # Save for future use
        with open(storms_df_file, 'wb') as f:
            pickle.dump(storms_df, f)
        return storms_df

print("Shared utilities and configuration loaded")

---

# Barrier Closures

This section analyzes past closures of the Eastern Scheldt barrier. The analysis:

1. Loads barrier closure dates from an Excel file
2. Counts closures per water year (July 1 to July 1)
3. Calculates statistics (min, mean, max, total closures)
4. Creates a bar chart visualization


In [ ]:
#| label: setup-master1
#| include: true
# Configuration - years from 1986 to 2024
Y = list(range(1986, 2025))  # [1986, 1987, ..., 2024]

print(f"Analysis configuration:")
print(f"  Water years: {Y[0]}/{str(Y[0]+1)[2:]} to {Y[-1]}/{str(Y[-1]+1)[2:]} ({len(Y)} years)")

## Analysis


In [ ]:
#| label: load-closures
#| include: true

# 1. Observed closure data - Eastern Scheldt
data_dir = '../2_DATA/1_BARRIER_CLOSURES'
excel_file = os.path.join(data_dir, 'Eastern_Scheldt_Barrier_Past_Closures_2024.xlsx')
df = pd.read_excel(excel_file, sheet_name='Closures')

# Extract closure dates (date only) from column 2 (index 1)
closure_dates_col = df.iloc[:, 1]  # Column 2 (0-indexed)

# Extract closure times (should be the 'Start of closure' column)
# Let's assume the time is in column 3 (index 2). Adjust if necessary.
if df.shape[1] > 2:
    closure_times_col = df.iloc[:, 3]  # Column 4 (Start of closure TIME/hh:mm if present)
else:
    # If there is no time column, fallback to NaT
    closure_times_col = [pd.NaT] * len(closure_dates_col)

# Combine closure date and time to make closure datetime
OCD = []
for d, t in zip(closure_dates_col, closure_times_col):
    if pd.isna(d):
        continue
    date_val = d.to_pydatetime() if isinstance(d, pd.Timestamp) else d
    if not pd.isna(t):
        # t can be a time or datetime or string like "13:00"
        if isinstance(t, pd.Timestamp):
            time_val = t.time()
        elif isinstance(t, str):
            try:
                time_val = pd.to_datetime(t).time()
            except Exception:
                time_val = None
        elif hasattr(t, 'time'):
            time_val = t.time()
        else:
            time_val = None
        if time_val is not None:
            combined_dt = datetime.combine(date_val.date(), time_val)
        else:
            combined_dt = date_val
    else:
        combined_dt = date_val
    OCD.append(combined_dt)

print(f"Loaded {len(OCD)} closure datetimes")
if len(OCD) > 0:
    print(f"  First closure: {OCD[0]}")
    print(f"  Last closure: {OCD[-1]}")

# 2. Count closures per water year (July 1 to July 1)
OCS = []  # Will store [index, year, year+1, num_closures]
YT = []   # Will store year labels like "1986/87"

for co, y in enumerate(Y, start=1):
    # Find closures between July 1 of year y and July 1 of year y+1
    start_date = datetime(y, 7, 1, 0, 0, 0)
    end_date = datetime(y + 1, 7, 1, 0, 0, 0)
    
    # Count closures in this water year
    closures_in_year = [d for d in OCD if start_date <= d < end_date]
    num_closures = len(closures_in_year)
    
    OCS.append([co, y, y + 1, num_closures])
    
    # Create year label (e.g., "1986/87")
    year_str = str(y + 1)
    YT.append(f"{y}/{year_str[2:]}")

OCS = np.array(OCS)

print(f"\nClosures per water year calculated for {len(OCS)} years")

print(f"  Total closures: {np.sum(OCS[:, 3])}")

# 3. Calculate statistics
E = np.array([
    np.min(OCS[:, 3]),      # Minimum closures per year
    np.mean(OCS[:, 3]),     # Mean closures per year
    np.max(OCS[:, 3]),      # Maximum closures per year
    np.sum(OCS[:, 3])       # Total closures
])

print(f"\nClosure statistics:")
print(f"  Min per year: {E[0]:.1f}")
print(f"  Mean per year: {E[1]:.2f}")
print(f"  Max per year: {E[2]:.1f}")
print(f"  Total: {E[3]:.0f}")

#| label: save-master1
#| include: true

# 4. Save data
output_file = os.path.join(output_dir, 'mast1.pkl')
with open(output_file, 'wb') as f:
    pickle.dump({
        'OCD': OCD,
        'OCS': OCS,
        'E': E,
        'YT': YT
    }, f)
print(f"\nData saved to {output_file}")

## Visualization 

This bar chart shows the number of barrier closures per water year from 1986/87 to 2024/25. The Eastern Scheldt barrier closes when water levels exceed a threshold, typically during storm surge events.


In [ ]:
#| label: plot-closures
#| fig-cap: Number of Eastern Scheldt barrier closures per water year (July 1 to June 30), 1986-2024

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(OCS[:, 0], OCS[:, 3], color='black')
ax.set_xlim(0.2, len(OCS) + 0.8)
ax.set_xticks(range(1, len(OCS) + 1))
ax.set_xticklabels(YT, rotation=90)
ax.set_ylim(0, 5)
ax.set_yticks(range(6))
ax.set_ylabel('Number of closures', fontweight='bold', fontsize=18)
ax.set_title('Eastern Scheldt', fontweight='bold', fontsize=18)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=16)

plt.tight_layout()
plt.show()

# Save figure
fig_file = os.path.join(output_dir, 'master1_closures.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"\nFigure saved to {fig_file}")

The following table lists all barrier closure dates, which correspond to storm events that triggered the Eastern Scheldt barrier.


In [ ]:
#| label: table-storms
#| tbl-cap: Complete list of Eastern Scheldt barrier closures (storm events), 1986-2024

# Get or create storms DataFrame
storms_df = get_or_create_storms_df()

# Save storms_df to mast1.pkl for consistency (update existing file)
mast1_file = os.path.join(output_dir, 'mast1.pkl')
if os.path.exists(mast1_file):
    with open(mast1_file, 'rb') as f:
        mast1_data = pickle.load(f)
    mast1_data['storms_df'] = storms_df
    with open(mast1_file, 'wb') as f:
        pickle.dump(mast1_data, f)

# Display table
print(f"\nTotal number of closures: {len(storms_df)}")
storms_df

---

# Tidal Harmonic Analysis

This section performs comprehensive tidal harmonic analysis on tide gauge data from the Eastern Scheldt, including data processing, harmonic decomposition, and visualization. The analysis:

1. Loads and processes raw tide gauge data from multiple sources (OS4 and RPBU gauges)
2. Performs harmonic tidal decomposition using `pytides`
3. Generates tidal predictions for all years
4. Compares observed water levels with predicted astronomical tides
5. Visualizes predicted high waters and barrier closures

The predicted tides represent only the astronomical component, while observed water levels include both tides and meteorological effects (storm surges, wind setup, etc.).

## Tide Gauge Data Processing

This section loads and processes raw tide gauge data from the Eastern Scheldt. The analysis:

1. Loads data from multiple tide gauge files (RPBU and OS4 gauges)
2. Cleans data by removing invalid values
3. Combines time series from different gauges to create a continuous record
4. Creates a visualization of the complete water level time series

The combined time series spans from 1986 to 2023, using OS4 gauge data for the early period (1986-1987) and RPBU gauge data for the main period (1987-2023).


In [ ]:
#| label: setup-master2
#| include: true

# Configuration
data_dir = '../2_DATA'

print(f"Data directory: {data_dir}")
print(f"Output directory: {output_dir}")

### Load RPBU 1987-2022 Data

Load the main tide gauge dataset from the RPBU gauge covering April 1987 to December 2022.


In [ ]:
#| label: load-rpbu-1987-2022
#| include: true

# 1. Load in Data 1987 to 2022 (RPBU gauge)
file_in = os.path.join(data_dir, '2_TIDE_GAUGE/ES/RPBU_1987_2022.dat')
D = np.loadtxt(file_in)

# Create time series: April 15, 1987 to December 31, 2022, 10-minute intervals
start_date = datetime(1987, 4, 15, 0, 0, 0)
end_date = datetime(2022, 12, 31, 23, 50, 0)
TS1 = pd.date_range(start=start_date, end=end_date, freq='10min').to_pydatetime()

# Extract water level data from column 3 (index 2)
WL1 = D[:, 2]

# Remove default values (9999 indicates missing)
WL1[WL1 == 9999] = np.nan

# Convert to meters (from cm)
WL1 = WL1 / 100.0

print(f"Loaded RPBU 1987-2022: {len(TS1):,} data points")
print(f"  Start: {TS1[0]}")
print(f"  End: {TS1[-1]}")
print(f"  Valid data: {np.sum(~np.isnan(WL1)):,} points ({100*np.sum(~np.isnan(WL1))/len(WL1):.1f}%)")

### Load RPBU 2023 Data

Load the 2023 data from the RPBU gauge to extend the time series.


In [ ]:
#| label: load-rpbu-2023
#| include: true

# 2. Load in Data 2023 (RPBU gauge)
file_in = os.path.join(data_dir, '2_TIDE_GAUGE/ES/RPBU_2023.dat')
D = np.loadtxt(file_in)

# Create time series: January 1, 2023 to December 31, 2023, 10-minute intervals
start_date = datetime(2023, 1, 1, 0, 0, 0)
end_date = datetime(2023, 12, 31, 23, 50, 0)
TS2 = pd.date_range(start=start_date, end=end_date, freq='10min').to_pydatetime()

# Extract water level data from column 3 (index 2)
WL2 = D[:, 2]

# Remove default values (9999 indicates missing)
WL2[WL2 == 9999] = np.nan

# Convert to meters (from cm)
WL2 = WL2 / 100.0

print(f"Loaded RPBU 2023: {len(TS2):,} data points")
print(f"  Start: {TS2[0]}")
print(f"  End: {TS2[-1]}")
print(f"  Valid data: {np.sum(~np.isnan(WL2)):,} points ({100*np.sum(~np.isnan(WL2))/len(WL2):.1f}%)")

### Load OS4 1982-2023 Data

Load data from the OS4 gauge, which provides coverage for the early period before RPBU data begins.


In [ ]:
#| label: load-os4
#| include: true

# 3. Load in Data 1982 to 2023 for OS4 gauge
file_in = os.path.join(data_dir, '2_TIDE_GAUGE/ES/OS4_1982_2023.dat')
D = np.loadtxt(file_in)

# Create time series: January 1, 1982 to December 31, 2023, 10-minute intervals
start_date = datetime(1982, 1, 1, 0, 0, 0)
end_date = datetime(2023, 12, 31, 23, 50, 0)
TSO = pd.date_range(start=start_date, end=end_date, freq='10min').to_pydatetime()

# Extract water level data from column 3 (index 2)
WLO = D[:, 2]

# Remove default values (values outside [-500, 500] are invalid)
WLO[(WLO < -500) | (WLO > 500)] = np.nan

# Convert to meters (from cm)
WLO = WLO / 100.0

print(f"Loaded OS4 1982-2023: {len(TSO):,} data points")
print(f"  Start: {TSO[0]}")
print(f"  End: {TSO[-1]}")
print(f"  Valid data: {np.sum(~np.isnan(WLO)):,} points ({100*np.sum(~np.isnan(WLO))/len(WLO):.1f}%)")

### Combine Time Series

Combine the time series from different gauges to create a continuous record. Use OS4 data for the early period (1986-1987) before RPBU starts, then switch to RPBU data for the main period.


In [ ]:
#| label: combine-series
#| include: true

# 4. Combine time series
# Use OS4 data for period 1986-01-01 to 1987-04-15 (before RPBU starts)
start_combine = datetime(1986, 1, 1, 0, 0, 0)
end_combine = datetime(1987, 4, 15, 0, 0, 0)
mask_early = (TSO >= start_combine) & (TSO < end_combine)
TSO_early = TSO[mask_early]
WLO_early = WLO[mask_early]

# Combine: OS4 early period + RPBU 1987-2022 + RPBU 2023
TSP = np.concatenate([TSO_early, TS1, TS2])
WLP = np.concatenate([WLO_early, WL1, WL2])

# Create reference time series for checking
TSCHECK = pd.date_range(
    start=datetime(1986, 1, 1, 0, 0, 0),
    end=datetime(2023, 12, 31, 23, 50, 0),
    freq='10min'
).to_pydatetime()

print(f"\nCombined time series: {len(TSP):,} data points")
print(f"  Start: {TSP[0]}")
print(f"  End: {TSP[-1]}")
print(f"  Valid data: {np.sum(~np.isnan(WLP)):,} points ({100*np.sum(~np.isnan(WLP))/len(WLP):.1f}%)")
print(f"  Time span: {(TSP[-1] - TSP[0]).days} days")

Save the combined time series to a pickle file for use in subsequent analyses.


In [ ]:
#| label: save-master2
#| include: true

# 5. Save data
output_file = os.path.join(output_dir, 'mast2.pkl')
with open(output_file, 'wb') as f:
    pickle.dump({
        'TSP': TSP,
        'WLP': WLP,
        'TSCHECK': TSCHECK
    }, f)
print(f"\nData saved to {output_file}")

### Visualization 

This figure shows the combined tide gauge water level time series from 1986 to 2023 at 10-minute intervals. The data combines OS4 gauge measurements (1986-1987) and RPBU gauge measurements (1987-2023) for the Eastern Scheldt. Water levels are shown in meters relative to NAP (Normal Amsterdam Peil, the Dutch reference datum). The time series shows tidal variations, storm surges, and long-term water level patterns over the 38-year period.


In [ ]:
#| label: plot-time-series
#| fig-cap: Combined tide gauge water level time series for the Eastern Scheldt, 1986-2023. Data combines OS4 gauge (1986-1987) and RPBU gauge (1987-2023) measurements at 10-minute intervals.

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(TSP, WLP, 'b', linewidth=0.5)
ax.set_xlabel('Date', fontweight='bold', fontsize=20)
ax.set_ylabel('Water level (m NAP)', fontweight='bold', fontsize=20)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=16)

plt.tight_layout()
plt.show()

# Save figure
fig_file = os.path.join(output_dir, 'master2_tide_gauge.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"\nFigure saved to {fig_file}")

## Tidal Analysis

This section performs harmonic tidal analysis on tide gauge data from the Eastern Scheldt. The analysis:

1. Calculates data quality for each year (1986-2023)
2. Performs harmonic tidal decomposition using `pytides` (replacing MATLAB's `t_tide`)
3. Generates tidal predictions for all years
4. Compares observed water levels with predicted astronomical tides

The predicted tides represent only the astronomical component, while observed water levels include both tides and meteorological effects (storm surges, wind setup, etc.).

### Setup and Configuration


In [ ]:
#| label: setup-master3
#| include: true

from pytides.tide import Tide

# Configuration
Y = list(range(1986, 2024))  # Years 1986 to 2023
th = 60  # Data quality threshold (percentage)
lat = 51.64  # Latitude for tidal analysis

print(f"Analysis configuration:")
print(f"  Years: {Y[0]} to {Y[-1]} ({len(Y)} years)")
print(f"  Data quality threshold: {th}%")
print(f"  Latitude: {lat}°")

### Load Tide Gauge Data


In [ ]:
#| label: load-data-master3
#| include: true

print("Loading tide gauge data...")
mast2_file = os.path.join(output_dir, 'mast2.pkl')
if not os.path.exists(mast2_file):
    raise FileNotFoundError(
        f"Required file {mast2_file} not found. "
        "Please run Part 2 (Tide Gauge Data Processing) first to generate the required data file."
    )
with open(mast2_file, 'rb') as f:
    data = pickle.load(f)
    TSP = data['TSP']
    WLP = data['WLP']

# Convert to numpy arrays if needed
TSP = np.array(TSP)
WLP = np.array(WLP)

print(f"Loaded {len(TSP):,} data points")
print(f"  Start: {TSP[0]}")
print(f"  End: {TSP[-1]}")
print(f"  Valid data: {np.sum(~np.isnan(WLP)):,} points ({100*np.sum(~np.isnan(WLP))/len(WLP):.1f}%)")

### Calculate Data Quality Per Year


In [ ]:
#| label: data-quality
#| include: true

print("\nCalculating data quality per year...")
DQ = []

for y in Y:
    # Find data points in this calendar year (Jan 1 to Jan 1 next year)
    start_date = datetime(y, 1, 1, 0, 0, 0)
    end_date = datetime(y + 1, 1, 1, 0, 0, 0)
    mask = (TSP >= start_date) & (TSP < end_date)
    j = np.where(mask)[0]
    
    if len(j) == 0:
        quality = 0
    else:
        # Count non-NaN values
        k = np.where(~np.isnan(WLP[j]))[0]
        quality = (len(k) / len(j)) * 100
    
    # Initialize with 3 columns: [year, quality, reference_year]
    # reference_year will be filled in during tidal analysis
    DQ.append([y, quality, np.nan])

DQ = np.array(DQ)

# Display summary statistics
print(f"\nData quality summary:")
print(f"  Range: {np.min(DQ[:, 1]):.1f}% to {np.max(DQ[:, 1]):.1f}%")
print(f"  Mean: {np.mean(DQ[:, 1]):.1f}%")
print(f"  Years with quality >= {th}%: {np.sum(DQ[:, 1] >= th)}/{len(Y)}")

### Tidal Analysis and Prediction

For each target year, the analysis determines a reference year, extracts data, performs harmonic decomposition, and generates predictions.


In [ ]:
#| label: tidal-analysis
#| include: true

print("\nPerforming tidal analysis and prediction...")
TSP2 = []
TIP = []

for co, y in enumerate(Y):
    print(f"  Processing year {y} ({co+1}/{len(Y)})...")
    
    # Determine reference year
    if DQ[co, 1] >= th:
        # Use target year as reference
        yr = y
        DQ[co, 2] = yr
    else:
        # Find nearest year with quality >= threshold
        # Calculate distances from target year
        distances = np.abs(DQ[:, 0] - y)
        
        # Filter to only years with quality >= threshold
        good_mask = DQ[:, 1] >= th
        
        if np.any(good_mask):
            # Get distances for good years only
            good_distances = distances[good_mask]
            good_indices = np.where(good_mask)[0]
            
            # Find nearest good year
            nearest_idx = good_indices[np.argmin(good_distances)]
            yr = int(DQ[nearest_idx, 0])
        else:
            # Fallback: use year with best quality
            best_idx = np.argmax(DQ[:, 1])
            yr = int(DQ[best_idx, 0])
        
        DQ[co, 2] = yr
    
    # Extract reference year data (1 year + 1 day for analysis)
    ref_start = datetime(yr, 1, 1, 0, 0, 0)
    ref_end = datetime(yr + 1, 1, 2, 0, 0, 0)  # +1 day
    
    mask_ref = (TSP >= ref_start) & (TSP < ref_end)
    j_ref = np.where(mask_ref)[0]
    
    if len(j_ref) == 0:
        print(f"    Warning: No data found for reference year {yr}")
        continue
    
    # Get water levels and times for reference year
    WLP_ref = WLP[j_ref]
    TSP_ref = TSP[j_ref]
    
    # Remove NaN values for pytides analysis
    valid_mask = ~np.isnan(WLP_ref)
    WLP_clean = WLP_ref[valid_mask]
    TSP_clean = TSP_ref[valid_mask]
    
    if len(WLP_clean) < 1000:  # Need sufficient data for analysis
        print(f"    Warning: Insufficient data for reference year {yr} ({len(WLP_clean)} points)")
        continue
    
    # Perform tidal analysis using pytides
    try:
        tide_model = Tide.decompose(
            heights=WLP_clean,
            t=TSP_clean
        )
        print(f"    Analyzed {len(WLP_clean):,} points from reference year {yr}")
    except Exception as e:
        print(f"    Error in tidal analysis for year {y}: {e}")
        continue
    
    # Generate prediction timestamps for target year (10-minute intervals)
    pred_start = datetime(y, 1, 1, 0, 0, 0)
    pred_end = datetime(y, 12, 31, 23, 50, 0)
    tsp2 = pd.date_range(start=pred_start, end=pred_end, freq='10min').to_pydatetime()
    
    # Predict tides
    try:
        tip = tide_model.at(tsp2)
        TIP.extend(tip)
        TSP2.extend(tsp2)
        print(f"    Predicted {len(tip):,} points for year {y}")
    except Exception as e:
        print(f"    Error in prediction for year {y}: {e}")
        continue

# Convert to numpy arrays
TSP2 = np.array(TSP2)
TIP = np.array(TIP)

print(f"\nTotal predictions: {len(TIP):,} points")
if len(TIP) > 0:
    print(f"  Start: {TSP2[0]}")
    print(f"  End: {TSP2[-1]}")

In [ ]:
#| label: save-master3
#| include: true

output_file = os.path.join(output_dir, 'mast3.pkl')
with open(output_file, 'wb') as f:
    pickle.dump({
        'TSP': TSP,
        'WLP': WLP,
        'TIP': TIP,
        'TSP2': TSP2,
        'DQ': DQ
    }, f)
print(f"Data saved to {output_file}")

### Visualization 

This figure compares observed water levels (blue) with predicted astronomical tides (red) from harmonic analysis. The predicted tides represent the astronomical component only, while observed water levels include both tides and meteorological effects.


In [ ]:
#| label: plot-comparison
#| fig-cap: Comparison of observed water levels (blue) and predicted astronomical tides (red) for the Eastern Scheldt, 1986-2023

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(TSP2, TIP, 'r', linewidth=0.5, label='Predicted tide', alpha=0.7)
ax.plot(TSP, WLP, 'b', linewidth=0.5, label='Observed water level', alpha=0.7)
ax.set_xlabel('Date', fontweight='bold', fontsize=16)
ax.set_ylabel('Level (m NAP)', fontweight='bold', fontsize=16)
ax.legend(fontsize=14)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

# Save figure
fig_file = os.path.join(output_dir, 'master3_tidal_analysis.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"\nFigure saved to {fig_file}")

## Predicted High Waters and Barrier Closures

This section plots time-series of predicted high waters and barrier closures for the Eastern Scheldt. The analysis:

1. Loads barrier closure dates and tide gauge data from previous analyses
2. Creates a visualization showing:
   - Barrier closure dates as vertical dashed lines
   - Predicted astronomical tides (TIP)
   - Observed water levels (WLP)
   - The difference between observed and predicted levels (surge component)

This visualization helps identify when barrier closures occurred relative to predicted high waters and actual water levels.


In [ ]:
#| label: load-data-master4
#| include: true

# Load all data using shared function
print("Loading data...")
data = load_all_data()
OCD = data['OCD']
TSP = data['TSP']
TIP = data['TIP']
WLP = data['WLP']

print(f"\nData loaded:")
print(f"  Closure dates: {len(OCD)} closures")
if len(OCD) > 0:
    print(f"    First closure: {OCD[0]}")
    print(f"    Last closure: {OCD[-1]}")
print(f"  Time series points: {len(TSP):,}")
print(f"  Predicted tides: {len(TIP):,}")
print(f"  Observed water levels: {len(WLP):,}")

## Visualization 

This figure shows predicted astronomical tides (red), observed water levels (blue), the difference between them (green), and barrier closure dates (vertical dashed magenta lines). The difference (WLP-TIP) represents the non-tidal component, primarily storm surge.


In [ ]:
#| label: plot-predicted-high-waters
#| fig-cap: Predicted high waters, observed water levels, and barrier closures for the Eastern Scheldt

fig, ax = plt.subplots(figsize=(14, 8))

# Plot vertical lines for barrier closure dates
for i in range(len(OCD)):
    ax.axvline(OCD[i], color='m', linestyle='--', linewidth=2, alpha=0.7)

# Plot predicted tides (TIP)
ax.plot(TSP, TIP, 'r', linewidth=1, label='Predicted tide (TIP)', alpha=0.8)

# Plot observed water levels (WLP)
ax.plot(TSP, WLP, 'b', linewidth=1, label='Observed water level (WLP)', alpha=0.8)

# Plot difference (WLP - TIP) - surge component
ax.plot(TSP, WLP - TIP, 'g', linewidth=1, label='Surge (WLP - TIP)', alpha=0.8)

# Formatting
ax.grid(True, alpha=0.3)
ax.set_xlabel('Date', fontweight='bold', fontsize=16)
ax.set_ylabel('Level (m)', fontweight='bold', fontsize=16)
ax.set_ylim(-3, 4)
ax.legend(fontsize=14, loc='best')
ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

# Save figure
fig_file = os.path.join(output_dir, 'master4_predicted_high_waters.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"\nFigure saved to {fig_file}")

## Detailed Storm Water Level Analysis

This section provides a detailed analysis of water levels and surge for Storm 5, Storm 6, Storm 7, Storm 8, Storm 9, Storm 10, Storm 11, Storm 12, Storm 13, Storm 18 (2007), and Storm 19 for the Eastern Scheldt, converting the MATLAB script `master5.m` to Python. The analysis:

1. Loads barrier closure dates and tide gauge data from previous analyses
2. Calculates surge component (WLP - TIP)
3. Creates detailed water level visualizations with closure date markers for each storm:
   - Zoomed water level and surge time series with closure markers (5-day window)
   - Detailed two-panel view showing water level vs. tide and surge separately

This provides a comprehensive view of the water level conditions that led to the barrier closures for Storms 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, and 19.

## Setup


In [ ]:
#| label: load-closure-tide-data
#| include: true

# Load all data using shared functions
print("Loading data...")
data = load_all_data()
OCD = data['OCD']
TSP = data['TSP']
TIP = data['TIP']
WLP = data['WLP']

# Calculate surge
SUP = WLP - TIP

# Get storms DataFrame
storms_df = get_or_create_storms_df()

print(f"\nData loaded:")
print(f"  Total closure dates: {len(OCD)} closures")
print(f"  Time series points: {len(TSP):,}")

# Spatial Surge and Meteorological Maps

This document provides spatial analysis of surge patterns and meteorological conditions for Storm 8 (closure event index 11), Storm 18 (closure event index 23, 2007), and Storm 19 (closure event index 24) for the Eastern Scheldt, converting the MATLAB script `master6.m` to Python. The analysis:

1. Loads barrier closure dates to identify the storm events
2. Loads GTSM reanalysis surge data (spatial surge patterns) for each storm
3. Loads ERA5 meteorological reanalysis data (pressure and wind fields) for each storm
4. Creates multiple visualizations for each storm:
   - GTSM surge maps at 6 time steps around the closure
   - ERA5 meteorological maps (pressure + wind vectors) at 6 time steps

This provides a comprehensive view of the spatial patterns and meteorological conditions that led to the barrier closures for Storms 8, 18, and 19.


In [ ]:
#| label: setup-maps
#| include: true

import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec

print(f"Analysis configuration:")
print(f"  Storms to process: Storm 8 (e=11, prefix '8_'), Storm 18 (e=23, prefix '18_'), and Storm 19 (e=24, prefix '19_')")
print(f"  GTSM time interval: {TINT} hours")
print(f"  Meteorological time interval: {TINT_MET} hours")
print(f"  Output directory: {output_dir}")

In [ ]:
#| label: load-closure-data
#| include: true

# Load barrier closure data using shared function
OCD = load_master1_data()
OCD = np.array(OCD)

# Validate event indices for all storms
e_storm8 = 11
e_storm18 = 23
e_storm19 = 24
if e_storm8 >= len(OCD):
    raise ValueError(f"Event index {e_storm8} (Storm 8) is out of range. There are {len(OCD)} closure events (indices 0-{len(OCD)-1})")
if e_storm18 >= len(OCD):
    raise ValueError(f"Event index {e_storm18} (Storm 18) is out of range. There are {len(OCD)} closure events (indices 0-{len(OCD)-1})")
if e_storm19 >= len(OCD):
    raise ValueError(f"Event index {e_storm19} (Storm 19) is out of range. There are {len(OCD)} closure events (indices 0-{len(OCD)-1})")

print(f"\nData loaded:")
print(f"  Total closure dates: {len(OCD)} closures")
print(f"  Storm 8 closure date: {OCD[e_storm8]}")
print(f"  Storm 18 closure date: {OCD[e_storm18]}")
print(f"  Storm 19 closure date: {OCD[e_storm19]}")

# Storms 

## Storm 1

Storm 1 uses datasets with prefix "1_" in the 2_DATA directory. The storm number corresponds directly to the row index in the closure dates table.

### Water Level and Surge Analysis


In [ ]:
#| label: plot-predicted-high-waters-zoomed-storm1
#| fig-cap: Predicted high waters, observed water levels, and barrier closure (zoomed view) for Storm 1 - Eastern Scheldt

# Load data
data = load_all_data()
OCD = data['OCD']
TSP = data['TSP']
TIP = data['TIP']
WLP = data['WLP']
storms_df = get_or_create_storms_df()

# First storm: closure index 0, prefix '1_'
e = 0
closure_date = OCD[e]
closure_dates = [closure_date]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

fig, ax = plt.subplots(figsize=(14, 8))

# Plot vertical lines for closure date
ax.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')

# Plot predicted tides, observed water levels, and surge
ax.plot(TSP, TIP, 'r', linewidth=1, label='Predicted tide (TIP)', alpha=0.8)
ax.plot(TSP, WLP, 'b', linewidth=1, label='Observed water level (WLP)', alpha=0.8)
ax.plot(TSP, WLP - TIP, 'g', linewidth=1, label='Surge (WLP - TIP)', alpha=0.8)

# Set zoom window: 2 days before to 3 days after closure
ax.set_xlim(closure_date - timedelta(days=2), closure_date + timedelta(days=3))
ax.set_ylim(-3, 4)
ax.grid(True, alpha=0.3)
ax.set_xlabel('Date', fontweight='bold', fontsize=16)
ax.set_ylabel('Level (m)', fontweight='bold', fontsize=16)
ax.set_title('Storm 1 - Predicted High Waters (Zoomed View)', fontweight='bold', fontsize=18)
ax.legend(fontsize=14, loc='best')
ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm1_predicted_high_waters_zoomed.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-water-level-surge-storm1
#| fig-cap: Water level and surge time series for Storm 1 closure event

# First storm: closure index 0
e = 0
closure_date = OCD[e]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

# Calculate surge
SUP = WLP - TIP

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top panel: Water level and predicted tide
ax1.plot(TSP, WLP, 'b', linewidth=2, label='Water level')
ax1.plot(TSP, TIP, 'r', linewidth=2, label='Astronomical tide')
ax1.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax1.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')
ax1.set_ylabel('Water level (m)', fontweight='bold', fontsize=20)
ax1.set_title('Storm 1 - Water Level and Surge', fontweight='bold', fontsize=18)
ax1.legend(fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=16)
ax1.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

# Bottom panel: Surge
surge_color = np.array([255, 103, 40]) / 255
ax2.plot(TSP, SUP, color=surge_color, linewidth=2)
ax2.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.set_xlabel('Date', fontweight='bold', fontsize=20)
ax2.set_ylabel('Surge (m)', fontweight='bold', fontsize=20)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=16)
ax2.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm1_water_level_surge.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

### Spatial Surge and Meteorological Maps


In [ ]:
#| label: load-spatial-data-storm1
#| include: true

# Load data
OCD = load_master1_data()
OCD = np.array(OCD)

# First storm: closure index 0, prefix '1_'
e = 0
prefix = '1_'
closure_date = OCD[e]
closure_year = closure_date.year
closure_month = closure_date.month
gtsm_start_date = closure_date
met_start_date = closure_date - timedelta(hours=6)

# Load GTSM reanalysis surge data
file_in = os.path.join(data_dir, '3_CODEC_GTSM', f'{prefix}reanalysis_surge_hourly_{closure_year}_{closure_month:02d}_v3.nc')
ds_gtsm = xr.open_dataset(file_in)

TS_GT = pd.to_datetime(ds_gtsm['time'].values)
X_GT = ds_gtsm['station_x_coordinate'].values
Y_GT = ds_gtsm['station_y_coordinate'].values
SU_GT = ds_gtsm['surge'].values

mask = (X_GT >= XB_GTSM[0]) & (X_GT <= XB_GTSM[1]) & (Y_GT >= YB_GTSM[0]) & (Y_GT <= YB_GTSM[1])
X_GT = X_GT[mask]
Y_GT = Y_GT[mask]
SU_GT = SU_GT[:, mask]

# Load ERA5 meteorological data
filein_met = os.path.join(data_dir, '4_ERA5', f'{prefix}ERA5_{closure_year}_{closure_month:02d}.nc')
ds_met = xr.open_dataset(filein_met)

LON = ds_met['longitude'].values
LAT = ds_met['latitude'].values
P_MET = ds_met['msl'].values
U_MET = ds_met['u10'].values
V_MET = ds_met['v10'].values
TS_MET = pd.to_datetime(ds_met['valid_time'].values)

# Reorder to (lon, lat, time)
dims = ds_met['msl'].dims
if dims[0] == 'valid_time' and dims[1] == 'latitude' and dims[2] == 'longitude':
    P_MET = np.transpose(P_MET, (2, 1, 0))
    U_MET = np.transpose(U_MET, (2, 1, 0))
    V_MET = np.transpose(V_MET, (2, 1, 0))

X_MET, Y_MET = np.meshgrid(LON, LAT)
X_MET = X_MET.T
Y_MET = Y_MET.T

lon_mask = (LON >= XB_MET[0]) & (LON <= XB_MET[1])
lat_mask = (LAT >= YB_MET[0]) & (LAT <= YB_MET[1])
lon_idx = np.where(lon_mask)[0]
lat_idx = np.where(lat_mask)[0]

In [ ]:
#| label: find-time-indices-storm1
#| include: true

# Find time indices for GTSM plots
time_diffs = np.abs([(t - gtsm_start_date).total_seconds() for t in TS_GT])
t_gtsm_base = np.argmin(time_diffs)
t_gtsm = [t_gtsm_base, t_gtsm_base + TINT, t_gtsm_base + TINT*2, 
          t_gtsm_base + TINT*3, t_gtsm_base + TINT*4, t_gtsm_base + TINT*5]
t_gtsm = [max(0, min(t, len(TS_GT)-1)) for t in t_gtsm]

# Find time indices for meteorological plots
max_time_idx = P_MET.shape[2] - 1
valid_ts_met = TS_MET[:max_time_idx+1]
target_times = [met_start_date + timedelta(hours=h) for h in [0, 12, 24, 36, 48, 60]]
t_met = [np.argmin(np.abs([(t - target_time).total_seconds() for t in valid_ts_met])) for target_time in target_times]

In [ ]:
#| label: plot-gtsm-surge-storm1
#| fig-cap: GTSM reanalysis surge maps at 6 time steps around Storm 1 closure event

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3]) for i in range(6)]

for i, t_idx in enumerate(t_gtsm):
    ax = axes[i]
    scatter = ax.scatter(X_GT, Y_GT, c=SU_GT[t_idx, :], cmap='jet', s=15, vmin=-0.5, vmax=1.0)
    ax.set_xlim(XB_GTSM)
    ax.set_ylim(YB_GTSM)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.tick_params(labelsize=12)
    ax.set_title(TS_GT[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(scatter, cax=cbar_ax, orientation='vertical')
cbar.set_label('Surge (m)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm1_gtsm_surge.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-era5-met-storm1
#| fig-cap: ERA5 reanalysis pressure and wind fields at 6 time steps around Storm 1 closure event

int_skip = 3
X_sub = X_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
Y_sub = Y_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
X_vec = X_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]
Y_vec = Y_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3], projection=ccrs.PlateCarree()) for i in range(6)]

ims = []
for i, t_idx in enumerate(t_met):
    ax = axes[i]
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.8, color='white')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.5, color='white')
    
    P_sub = P_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1, t_idx] / 100
    U_vec = U_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    V_vec = V_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    
    im = ax.pcolormesh(X_sub, Y_sub, P_sub, cmap='jet', shading='gouraud', 
                       vmin=967, vmax=1020, transform=ccrs.PlateCarree())
    ims.append(im)
    ax.quiver(X_vec, Y_vec, U_vec, V_vec, color='k', scale=1200, width=0.002, 
              transform=ccrs.PlateCarree())
    
    ax.set_xlim(XB_MET)
    ax.set_ylim(YB_MET)
    ax.gridlines(draw_labels=False, linewidth=0.5, color='gray', alpha=0.3, linestyle='--')
    ax.tick_params(labelsize=12)
    ax.set_title(TS_MET[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(ims[0], cax=cbar_ax, orientation='vertical')
cbar.set_label('Pressure (hPa)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm1_era5_meteorology.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

## Storm 2

Storm 2 uses datasets with prefix "2_" in the 2_DATA directory. The storm number corresponds directly to the row index in the closure dates table.

### Water Level and Surge Analysis


In [ ]:
#| label: plot-predicted-high-waters-zoomed-storm2
#| fig-cap: Predicted high waters, observed water levels, and barrier closure (zoomed view) for Storm 2 - Eastern Scheldt

# Load data
data = load_all_data()
OCD = data['OCD']
TSP = data['TSP']
TIP = data['TIP']
WLP = data['WLP']

# Second storm: closure index 1, prefix '2_'
e = 1
closure_date = OCD[e]
closure_dates = [closure_date]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

fig, ax = plt.subplots(figsize=(14, 8))

# Plot vertical lines for closure date
ax.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')

# Plot predicted tides, observed water levels, and surge
ax.plot(TSP, TIP, 'r', linewidth=1, label='Predicted tide (TIP)', alpha=0.8)
ax.plot(TSP, WLP, 'b', linewidth=1, label='Observed water level (WLP)', alpha=0.8)
ax.plot(TSP, WLP - TIP, 'g', linewidth=1, label='Surge (WLP - TIP)', alpha=0.8)

# Set zoom window: 2 days before to 3 days after closure
ax.set_xlim(closure_date - timedelta(days=2), closure_date + timedelta(days=3))
ax.set_ylim(-3, 4)
ax.grid(True, alpha=0.3)
ax.set_xlabel('Date', fontweight='bold', fontsize=16)
ax.set_ylabel('Level (m)', fontweight='bold', fontsize=16)
ax.set_title('Storm 2 - Predicted High Waters (Zoomed View)', fontweight='bold', fontsize=18)
ax.legend(fontsize=14, loc='best')
ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm2_predicted_high_waters_zoomed.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-water-level-surge-storm2
#| fig-cap: Water level and surge time series for Storm 2 closure event

# Second storm: closure index 1
e = 1
closure_date = OCD[e]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

# Calculate surge
SUP = WLP - TIP

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top panel: Water level and predicted tide
ax1.plot(TSP, WLP, 'b', linewidth=2, label='Water level')
ax1.plot(TSP, TIP, 'r', linewidth=2, label='Astronomical tide')
ax1.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax1.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')
ax1.set_ylabel('Water level (m)', fontweight='bold', fontsize=20)
ax1.set_title('Storm 2 - Water Level and Surge', fontweight='bold', fontsize=18)
ax1.legend(fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=16)
ax1.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

# Bottom panel: Surge
surge_color = np.array([255, 103, 40]) / 255
ax2.plot(TSP, SUP, color=surge_color, linewidth=2)
ax2.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.set_xlabel('Date', fontweight='bold', fontsize=20)
ax2.set_ylabel('Surge (m)', fontweight='bold', fontsize=20)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=16)
ax2.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm2_water_level_surge.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

### Spatial Surge and Meteorological Maps


In [ ]:
#| label: load-spatial-data-storm2
#| include: true

# Load data
OCD = load_master1_data()
OCD = np.array(OCD)

# Second storm: closure index 1, prefix '2_'
e = 1
prefix = '2_'
closure_date = OCD[e]
closure_year = closure_date.year
closure_month = closure_date.month
gtsm_start_date = closure_date
met_start_date = closure_date - timedelta(hours=6)

# Load GTSM reanalysis surge data
file_in = os.path.join(data_dir, '3_CODEC_GTSM', f'{prefix}reanalysis_surge_hourly_{closure_year}_{closure_month:02d}_v3.nc')
ds_gtsm = xr.open_dataset(file_in)

TS_GT = pd.to_datetime(ds_gtsm['time'].values)
X_GT = ds_gtsm['station_x_coordinate'].values
Y_GT = ds_gtsm['station_y_coordinate'].values
SU_GT = ds_gtsm['surge'].values

mask = (X_GT >= XB_GTSM[0]) & (X_GT <= XB_GTSM[1]) & (Y_GT >= YB_GTSM[0]) & (Y_GT <= YB_GTSM[1])
X_GT = X_GT[mask]
Y_GT = Y_GT[mask]
SU_GT = SU_GT[:, mask]

# Load ERA5 meteorological data
filein_met = os.path.join(data_dir, '4_ERA5', f'{prefix}ERA5_{closure_year}_{closure_month:02d}.nc')
ds_met = xr.open_dataset(filein_met)

LON = ds_met['longitude'].values
LAT = ds_met['latitude'].values
P_MET = ds_met['msl'].values
U_MET = ds_met['u10'].values
V_MET = ds_met['v10'].values
TS_MET = pd.to_datetime(ds_met['valid_time'].values)

# Reorder to (lon, lat, time)
dims = ds_met['msl'].dims
if dims[0] == 'valid_time' and dims[1] == 'latitude' and dims[2] == 'longitude':
    P_MET = np.transpose(P_MET, (2, 1, 0))
    U_MET = np.transpose(U_MET, (2, 1, 0))
    V_MET = np.transpose(V_MET, (2, 1, 0))

X_MET, Y_MET = np.meshgrid(LON, LAT)
X_MET = X_MET.T
Y_MET = Y_MET.T

lon_mask = (LON >= XB_MET[0]) & (LON <= XB_MET[1])
lat_mask = (LAT >= YB_MET[0]) & (LAT <= YB_MET[1])
lon_idx = np.where(lon_mask)[0]
lat_idx = np.where(lat_mask)[0]

In [ ]:
#| label: find-time-indices-storm2
#| include: true

# Find time indices for GTSM plots
time_diffs = np.abs([(t - gtsm_start_date).total_seconds() for t in TS_GT])
t_gtsm_base = np.argmin(time_diffs)
t_gtsm = [t_gtsm_base, t_gtsm_base + TINT, t_gtsm_base + TINT*2, 
          t_gtsm_base + TINT*3, t_gtsm_base + TINT*4, t_gtsm_base + TINT*5]
t_gtsm = [max(0, min(t, len(TS_GT)-1)) for t in t_gtsm]

# Find time indices for meteorological plots
max_time_idx = P_MET.shape[2] - 1
valid_ts_met = TS_MET[:max_time_idx+1]
target_times = [met_start_date + timedelta(hours=h) for h in [0, 12, 24, 36, 48, 60]]
t_met = [np.argmin(np.abs([(t - target_time).total_seconds() for t in valid_ts_met])) for target_time in target_times]

In [ ]:
#| label: plot-gtsm-surge-storm2
#| fig-cap: GTSM reanalysis surge maps at 6 time steps around Storm 2 closure event

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3]) for i in range(6)]

for i, t_idx in enumerate(t_gtsm):
    ax = axes[i]
    scatter = ax.scatter(X_GT, Y_GT, c=SU_GT[t_idx, :], cmap='jet', s=15, vmin=-0.5, vmax=1.0)
    ax.set_xlim(XB_GTSM)
    ax.set_ylim(YB_GTSM)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.tick_params(labelsize=12)
    ax.set_title(TS_GT[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(scatter, cax=cbar_ax, orientation='vertical')
cbar.set_label('Surge (m)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm2_gtsm_surge.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-era5-met-storm2
#| fig-cap: ERA5 reanalysis pressure and wind fields at 6 time steps around Storm 2 closure event

int_skip = 3
X_sub = X_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
Y_sub = Y_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
X_vec = X_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]
Y_vec = Y_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3], projection=ccrs.PlateCarree()) for i in range(6)]

ims = []
for i, t_idx in enumerate(t_met):
    ax = axes[i]
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.8, color='white')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.5, color='white')
    
    P_sub = P_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1, t_idx] / 100
    U_vec = U_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    V_vec = V_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    
    im = ax.pcolormesh(X_sub, Y_sub, P_sub, cmap='jet', shading='gouraud', 
                       vmin=967, vmax=1020, transform=ccrs.PlateCarree())
    ims.append(im)
    ax.quiver(X_vec, Y_vec, U_vec, V_vec, color='k', scale=1200, width=0.002, 
              transform=ccrs.PlateCarree())
    
    ax.set_xlim(XB_MET)
    ax.set_ylim(YB_MET)
    ax.gridlines(draw_labels=False, linewidth=0.5, color='gray', alpha=0.3, linestyle='--')
    ax.tick_params(labelsize=12)
    ax.set_title(TS_MET[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(ims[0], cax=cbar_ax, orientation='vertical')
cbar.set_label('Pressure (hPa)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm2_era5_meteorology.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

## Storm 3

Storm 3 uses datasets with prefix "3_" in the 2_DATA directory. The storm number corresponds directly to the row index in the closure dates table.

### Water Level and Surge Analysis


In [ ]:
#| label: plot-predicted-high-waters-zoomed-storm3
#| fig-cap: Predicted high waters, observed water levels, and barrier closure (zoomed view) for Storm 3 - Eastern Scheldt

# Load data
data = load_all_data()
OCD = data['OCD']
TSP = data['TSP']
TIP = data['TIP']
WLP = data['WLP']

# Third storm: closure index 2, prefix '3_'
e = 2
closure_date = OCD[e]
closure_dates = [closure_date]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

fig, ax = plt.subplots(figsize=(14, 8))

# Plot vertical lines for closure date
ax.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')

# Plot predicted tides, observed water levels, and surge
ax.plot(TSP, TIP, 'r', linewidth=1, label='Predicted tide (TIP)', alpha=0.8)
ax.plot(TSP, WLP, 'b', linewidth=1, label='Observed water level (WLP)', alpha=0.8)
ax.plot(TSP, WLP - TIP, 'g', linewidth=1, label='Surge (WLP - TIP)', alpha=0.8)

# Set zoom window: 2 days before to 3 days after closure
ax.set_xlim(closure_date - timedelta(days=2), closure_date + timedelta(days=3))
ax.set_ylim(-3, 4)
ax.grid(True, alpha=0.3)
ax.set_xlabel('Date', fontweight='bold', fontsize=16)
ax.set_ylabel('Level (m)', fontweight='bold', fontsize=16)
ax.set_title('Storm 3 - Predicted High Waters (Zoomed View)', fontweight='bold', fontsize=18)
ax.legend(fontsize=14, loc='best')
ax.tick_params(labelsize=14)

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm3_predicted_high_waters_zoomed.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-water-level-surge-storm3
#| fig-cap: Water level and surge time series for Storm 3 closure event

# Third storm: closure index 2
e = 2
closure_date = OCD[e]
closure_dates_plus_one = [closure_date + timedelta(days=1)]

# Calculate surge
SUP = WLP - TIP

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top panel: Water level and predicted tide
ax1.plot(TSP, WLP, 'b', linewidth=2, label='Water level')
ax1.plot(TSP, TIP, 'r', linewidth=2, label='Astronomical tide')
ax1.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date')
ax1.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7, label='Closure date + 1 day')
ax1.set_ylabel('Water level (m)', fontweight='bold', fontsize=20)
ax1.set_title('Storm 3 - Water Level and Surge', fontweight='bold', fontsize=18)
ax1.legend(fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=16)
ax1.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

# Bottom panel: Surge
surge_color = np.array([255, 103, 40]) / 255
ax2.plot(TSP, SUP, color=surge_color, linewidth=2)
ax2.axvline(closure_date, color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.axvline(closure_dates_plus_one[0], color='m', linestyle='--', linewidth=2, alpha=0.7)
ax2.set_xlabel('Date', fontweight='bold', fontsize=20)
ax2.set_ylabel('Surge (m)', fontweight='bold', fontsize=20)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=16)
ax2.set_xlim(closure_date - timedelta(hours=12), closure_date + timedelta(hours=60))

plt.tight_layout()
plt.show()

fig_file = os.path.join(output_dir, 'high_waters_storm3_water_level_surge.png')
plt.savefig(fig_file, dpi=150, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

### Spatial Surge and Meteorological Maps


In [ ]:
#| label: load-spatial-data-storm3
#| include: true

# Load data
OCD = load_master1_data()
OCD = np.array(OCD)

# Third storm: closure index 2, prefix '3_'
e = 2
prefix = '3_'
closure_date = OCD[e]
closure_year = closure_date.year
closure_month = closure_date.month
gtsm_start_date = closure_date
met_start_date = closure_date - timedelta(hours=6)

# Load GTSM reanalysis surge data
file_in = os.path.join(data_dir, '3_CODEC_GTSM', f'{prefix}reanalysis_surge_hourly_{closure_year}_{closure_month:02d}_v3.nc')
ds_gtsm = xr.open_dataset(file_in)

TS_GT = pd.to_datetime(ds_gtsm['time'].values)
X_GT = ds_gtsm['station_x_coordinate'].values
Y_GT = ds_gtsm['station_y_coordinate'].values
SU_GT = ds_gtsm['surge'].values

mask = (X_GT >= XB_GTSM[0]) & (X_GT <= XB_GTSM[1]) & (Y_GT >= YB_GTSM[0]) & (Y_GT <= YB_GTSM[1])
X_GT = X_GT[mask]
Y_GT = Y_GT[mask]
SU_GT = SU_GT[:, mask]

# Load ERA5 meteorological data
filein_met = os.path.join(data_dir, '4_ERA5', f'{prefix}ERA5_{closure_year}_{closure_month:02d}.nc')
ds_met = xr.open_dataset(filein_met)

LON = ds_met['longitude'].values
LAT = ds_met['latitude'].values
P_MET = ds_met['msl'].values
U_MET = ds_met['u10'].values
V_MET = ds_met['v10'].values
TS_MET = pd.to_datetime(ds_met['valid_time'].values)

# Reorder to (lon, lat, time)
dims = ds_met['msl'].dims
if dims[0] == 'valid_time' and dims[1] == 'latitude' and dims[2] == 'longitude':
    P_MET = np.transpose(P_MET, (2, 1, 0))
    U_MET = np.transpose(U_MET, (2, 1, 0))
    V_MET = np.transpose(V_MET, (2, 1, 0))

X_MET, Y_MET = np.meshgrid(LON, LAT)
X_MET = X_MET.T
Y_MET = Y_MET.T

lon_mask = (LON >= XB_MET[0]) & (LON <= XB_MET[1])
lat_mask = (LAT >= YB_MET[0]) & (LAT <= YB_MET[1])
lon_idx = np.where(lon_mask)[0]
lat_idx = np.where(lat_mask)[0]

In [ ]:
#| label: find-time-indices-storm3
#| include: true

# Find time indices for GTSM plots
time_diffs = np.abs([(t - gtsm_start_date).total_seconds() for t in TS_GT])
t_gtsm_base = np.argmin(time_diffs)
t_gtsm = [t_gtsm_base, t_gtsm_base + TINT, t_gtsm_base + TINT*2, 
          t_gtsm_base + TINT*3, t_gtsm_base + TINT*4, t_gtsm_base + TINT*5]
t_gtsm = [max(0, min(t, len(TS_GT)-1)) for t in t_gtsm]

# Find time indices for meteorological plots
max_time_idx = P_MET.shape[2] - 1
valid_ts_met = TS_MET[:max_time_idx+1]
target_times = [met_start_date + timedelta(hours=h) for h in [0, 12, 24, 36, 48, 60]]
t_met = [np.argmin(np.abs([(t - target_time).total_seconds() for t in valid_ts_met])) for target_time in target_times]

In [ ]:
#| label: plot-gtsm-surge-storm3
#| fig-cap: GTSM reanalysis surge maps at 6 time steps around Storm 3 closure event

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3]) for i in range(6)]

for i, t_idx in enumerate(t_gtsm):
    ax = axes[i]
    scatter = ax.scatter(X_GT, Y_GT, c=SU_GT[t_idx, :], cmap='jet', s=15, vmin=-0.5, vmax=1.0)
    ax.set_xlim(XB_GTSM)
    ax.set_ylim(YB_GTSM)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.tick_params(labelsize=12)
    ax.set_title(TS_GT[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(scatter, cax=cbar_ax, orientation='vertical')
cbar.set_label('Surge (m)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm3_gtsm_surge.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")

In [ ]:
#| label: plot-era5-met-storm3
#| fig-cap: ERA5 reanalysis pressure and wind fields at 6 time steps around Storm 3 closure event

int_skip = 3
X_sub = X_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
Y_sub = Y_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1]
X_vec = X_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]
Y_vec = Y_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip]

fig = plt.figure(figsize=(16, 10))
gs = gridspec.GridSpec(2, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.3, hspace=0.25)
axes = [fig.add_subplot(gs[i // 3, i % 3], projection=ccrs.PlateCarree()) for i in range(6)]

ims = []
for i, t_idx in enumerate(t_met):
    ax = axes[i]
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.8, color='white')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=0.5, color='white')
    
    P_sub = P_MET[lon_idx[0]:lon_idx[-1]+1, lat_idx[0]:lat_idx[-1]+1, t_idx] / 100
    U_vec = U_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    V_vec = V_MET[lon_idx[0]:lon_idx[-1]+1:int_skip, lat_idx[0]:lat_idx[-1]+1:int_skip, t_idx]
    
    im = ax.pcolormesh(X_sub, Y_sub, P_sub, cmap='jet', shading='gouraud', 
                       vmin=967, vmax=1020, transform=ccrs.PlateCarree())
    ims.append(im)
    ax.quiver(X_vec, Y_vec, U_vec, V_vec, color='k', scale=1200, width=0.002, 
              transform=ccrs.PlateCarree())
    
    ax.set_xlim(XB_MET)
    ax.set_ylim(YB_MET)
    ax.gridlines(draw_labels=False, linewidth=0.5, color='gray', alpha=0.3, linestyle='--')
    ax.tick_params(labelsize=12)
    ax.set_title(TS_MET[t_idx].strftime('%Y-%m-%d %H:%M'), fontweight='bold', fontsize=16)
    if i >= 3:
        ax.set_xlabel('Longitude (deg)', fontweight='bold', fontsize=14)
    if i % 3 == 0:
        ax.set_ylabel('Latitude (deg)', fontweight='bold', fontsize=14)

cbar_ax = fig.add_subplot(gs[:, 3])
cbar = plt.colorbar(ims[0], cax=cbar_ax, orientation='vertical')
cbar.set_label('Pressure (hPa)', fontweight='bold', fontsize=14)
cbar.ax.tick_params(labelsize=12)

plt.tight_layout(rect=[0, 0, 0.97, 1])
plt.show()

fig_file = os.path.join(output_dir, 'maps_storm3_era5_meteorology.png')
plt.savefig(fig_file, dpi=100, bbox_inches='tight')
print(f"Figure saved to {fig_file}")